In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bz2
import csv
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split


In [ ]:


file_path = "/content/train.ft.txt.bz2"
test = '/content/test.ft.txt.bz2'
with bz2.open(file_path, "rt", encoding="utf-8") as f:
    training_data = [next(f).strip() for _ in range(12000)]

with bz2.open(test, "rt", encoding="utf-8") as fi:
    test_data = [next(fi).strip() for _ in range(12000)]

#df = pd.DataFrame(training_data, columns=["text"])
#print(df.head())

print(len(training_data))
print(len(test_data))


12000
12000


In [ ]:
labels = []
texts = []


for line in training_data:
    line = line.strip()
    match = re.match(r'__label__(\d)\s+(.*)', line)
    if match:
        labels.append(match.group(1))
        texts.append(match.group(2))




In [ ]:
training_labels = [int(re.match(r'__label__(\d+)', line).group(1)) for line in training_data if re.match(r'__label__(\d+)', line)]
training_texts  = [re.sub(r'__label__(\d+)\s*', '', line) for line in training_data]


test_labels = [int(re.match(r'__label__(\d+)', line).group(1)) for line in test_data if re.match(r'__label__(\d+)', line)]
test_texts  = [re.sub(r'__label__(\d+)\s*', '', line) for line in test_data]



In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer , PorterStemmer

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def clean_text(text):
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    #words = [ps.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

In [ ]:
training_texts = [clean_text(text) for text in training_texts]
test_texts = [clean_text(text) for text in test_texts]

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X = cv.fit_transform(training_texts)

x_test_cv = cv.transform(test_texts)
y = np.array(training_labels)

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tf = TfidfVectorizer()
x = tf.fit_transform(training_texts)
Y = np.array(training_labels)



x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
model2 = LogisticRegression()
model2.fit(x_train, y_train)

LogisticRegression()

In [ ]:
y_pred = model2.predict(x_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.8745833333333334


In [ ]:
print(confusion_matrix(y_test, y_pred))

[[1051  138]
 [ 163 1048]]
